In [10]:
# import necessary packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [17]:
# calculating Gaussian parameters using mle
def gaussian_mle(trainFile):
    # read the trainset file     
    fileData = open(trainFile, "r",encoding='utf-8-sig')
    lines = fileData.readlines()
    
    # create two matrices for two different classes    
    xsClass1 = []
    ysClass1 = []
    xsClass2 = []
    ysClass2 = []

    for line in lines:
        x = line.split()
        xs = float(x[0])
        ys = float(x[1])
        yc = int(x[2])
        if yc == 0:
            xsClass1.append(xs)
            ysClass1.append(ys)
        else:
            xsClass2.append(xs)
            ysClass2.append(ys)
     
    # for class0
    matClass1 = np.zeros((len(xsClass1), 2))
    for i in range(matClass1.shape[0]):
        matClass1[i][0] = xsClass1[i]
    for i in range(matClass1.shape[0]):
        matClass1[i][1] = ysClass1[i]
    
    # for class1
    matClass2 = np.zeros((len(xsClass2), 2))
    for i in range(matClass2.shape[0]):
        matClass2[i][0] = xsClass2[i]
    for i in range(matClass2.shape[0]):
        matClass2[i][1] = ysClass2[i]

    # find mean vector for class0
    m = np.zeros((len(xsClass1), 2))
    for i in range(len(matClass1)):
        m[i][0] = matClass1[i][0]
        m[i][1] = matClass1[i][1]
    mu1 = m.sum(axis = 0)/(len(xsClass1)) 
    mean1 = np.matrix(mu1).T
    
    # find mean vector for class1
    m1 = np.zeros((len(xsClass2), 2))
    for i in range(len(matClass2)):
        m1[i][0] = matClass2[i][0]
        m1[i][1] = matClass2[i][1]
    mu2 = m1.sum(axis = 0)/(len(xsClass2)) 
    mean2 = np.matrix(mu2).T
    
    # find covariance matrices for class0
    sigmaClass1 = np.zeros((len(xsClass1), 2))
    cov1 = np.zeros((len(xsClass1), 2))

    for i in range(matClass1.shape[0]):
        cov1[i][0] = matClass1[i][0]-mu1[0]
        cov1[i][1] = matClass1[i][1]-mu1[1]
        transposecov1 = np.matrix(cov1).T
        sigmaClass1 = transposecov1*cov1/matClass1.shape[0]

    # find covariance matrices for class1
    sigmaClass2 = np.zeros((len(xsClass2), 2))
    cov2 = np.zeros((len(xsClass2), 2))

    for i in range(matClass2.shape[0]):
        cov2[i][0] = matClass2[i][0]-mu2[0]
        cov2[i][1] = matClass2[i][1]-mu2[1]
        transposecov2 = np.matrix(cov2).T
        sigmaClass2 = transposecov2*cov2/matClass2.shape[0]   
        
    return mean1, mean2, sigmaClass1, sigmaClass2

# uncomment this line for output
# gaussian_mle("SYNTH.TR")

In [18]:
# create three dictionaries to store the important results of three cases for further visualizations
bestAccuracy_dict1 = dict()
bestAccuracy_dict2 = dict()
bestAccuracy_dict3 = dict()
predictedMatrix_dict = dict()  

# uncomment to clear the dictionary if necessary
# bestAccuracy_dict1.clear()
# bestAccuracy_dict2.clear()
# bestAccuracy_dict3.clear()

In [19]:
mean1,mean2,sigmaClass1,sigmaClass2  = gaussian_mle("SYNTH.TR")

# calculate for different probabilities and create dictionary with probability as keys and accuracy as values
def mpp(testFile, probClass1,mean1,mean2,sigma1,sigma2,sigmaSqr):
    probClass2 = 1-probClass1
    
    # read the test data file
    fileData1 = open(testFile, "r",encoding='utf-8-sig')
    liness = fileData1.readlines()
    
    # create a matrix from test data for calculation
    testSet = np.zeros((len(liness),3))
    xst = []
    yst = []
    yct = []
    for line in liness:
        xt = line.split()
        xst.append(float(xt[0]))
        yst.append(float(xt[1]))
        yct.append(int(xt[2]))
    for i in range(len(liness)):
        testSet[i][0] = xst[i]
        testSet[i][1] = yst[i]
        testSet[i][2] = yct[i]
 
    # find covariance matrices for CASE I
    I = np.identity(2)
    sigma3 = sigmaSqr*I

    # calculate determinant and inverse of new covariance Matrix (case 1: covariance matrices are equal to (sigma^2)I)
    detClass1 = np.linalg.det(sigma3)
    detClass2 = detClass1
    invClass1 = np.linalg.inv(sigma3)
    invClass2 = invClass1

    # test the three cases of discriminant analysis for best accuracy
    LAMBDA11 = 0
    LAMBDA12 = 1
    LAMBDA21 = 1
    LAMBDA22 = 0

    testCase = 0
    while testCase < 3:
        bestLambda1 = 1.0
        bestLambda2 = 1.0
        bestAccuracy = 0

        lambdaClass1 = 1.0
        while lambdaClass1 > 0:
            bestLambda1 = 1.0
            bestLambda2 = 1.0
            bestAccuracy = 0

            lambdaClass2 = 1.0
            while lambdaClass2 > 0:
                predictedMatrix = np.zeros((len(testSet), 2))
                correctGuesses = 0
                line1 = np.zeros((1, 2))
                for i in range(len(testSet)):
                    line1[:,0] = testSet[i][0]
                    line1[:,1] = testSet[i][1]
                    line2 = line1.T
                    mahalanobis = line2 - mean1
                    mahalanobis1 = mahalanobis.T
                    mahalanobis2 = mahalanobis1 * invClass1
                    mahalanobis3 = mahalanobis2 * (line2 - mean1)
                    varMahalanobis = float(-0.5 * mahalanobis3[0][0])
                    probIsClass1 = float((1.0 / math.sqrt(2 * math.pi * detClass1)) * math.exp(varMahalanobis) * (probClass1))

                    mahalanobiss = line2 - mean2
                    mahalanobiss1 = mahalanobiss.T
                    mahalanobiss2 = mahalanobiss1 * invClass2
                    mahalanobiss3 = mahalanobiss2 * (line2 - mean2)
                    varMahalanobiss = float(-0.5 * mahalanobiss3[0][0])

                    probIsClass2 = float((1.0 / math.sqrt(2 * math.pi * detClass2)) * math.exp(varMahalanobiss) * (probClass2))

                    aux = float(probIsClass1)
                    probIsClass1 = lambdaClass1 * probIsClass1 + (1 - lambdaClass1) * probIsClass2
                    probIsClass2 = lambdaClass2 * probIsClass2 + (1 - lambdaClass2) * aux

                    predictedClass = 0
                    if probIsClass2 > probIsClass1:
                        predictedClass = 1

                    if predictedClass == testSet[i][2]: 
                        correctGuesses+=1

                    error = float(min(probIsClass2, probIsClass1))

                    predictedMatrix[i][1] = error
                    predictedMatrix[i][0] = predictedClass

                acc = float(correctGuesses) / float(len(testSet))
                if acc > bestAccuracy:
                    bestAccuracy = acc
                    bestLambda1 = lambdaClass1
                    bestLambda2 = lambdaClass2
                lambdaClass2 -= 0.05
            lambdaClass1 -= 0.05

        if testCase == 0:
            print ("Discriminant Function Case 1 (features independents): accuracy = ",bestAccuracy,
                   "with probability",probClass1,"and",probClass2, "Lambda1,1=",bestLambda1,";Lambda1,2=",
                   1 - bestLambda1,";Lambda2,1=",bestLambda2,";Lambda2,2=",(1 - bestLambda2))
            acc1 = bestAccuracy

            # calculate determinant and inverse of new covariance matrices (case 2: covariance matrices is equal)
            detClass1 = np.linalg.det(sigma1)
            detClass2 = detClass1
            invClass1 = np.linalg.inv(sigma1)
            invClass2 = invClass1

        elif testCase == 1:
            print ("Discriminant Function Case 2 (covariance matrix equal for all features): accuracy = ",
                   bestAccuracy,"with probability",probClass1,"and",probClass2," Lambda1,1=",bestLambda1,
                   ";Lambda1,1=",bestLambda1,";Lambda1,2=",1-bestLambda1,";Lambda2,1=",bestLambda2,";Lambda2,2=",
                   (1 - bestLambda2))
            acc2 = bestAccuracy

            # calculate determinant and inverse of new covariance matrices (case 3: covariance matrices are different)
            detClass1 = np.linalg.det(sigma1)
            invClass1 = np.linalg.inv(sigma1)
            invClass2 = np.linalg.inv(sigma2)
            detClass2 = np.linalg.det(sigma2)

        elif testCase == 2:
            print ("Discriminant Function Case 3 (arbitrary): accuracy = ", bestAccuracy,
                   "with probability",probClass1,"and",probClass2," Lambda1,1=",bestLambda1 , ";Lambda1,2=" 
                   , (1 - bestLambda1),";Lambda2,1=" , bestLambda2 ,";Lambda2,2=", (1 - bestLambda2))
            acc3 = bestAccuracy

        testCase+=1
    bestAccuracy_dict1[probClass1]= acc1
    bestAccuracy_dict2[probClass1]= acc2
    bestAccuracy_dict3[probClass1]= acc3
    return bestAccuracy_dict1,bestAccuracy_dict2,bestAccuracy_dict3

# uncomment this line for output
# mpp("SYNTH.TE", 0.5,mean1,mean2,sigmaClass1,sigmaClass2,sigmaClass1[1,1])

In [20]:
# kept the result for convenience
bestAccuracy_dict1={0.1: 0.735, 0.2: 0.723, 0.3: 0.708, 0.4: 0.704, 0.5: 0.713, 0.6: 0.714, 0.7: 0.709, 0.8: 0.714, 0.9: 0.706}
bestAccuracy_dict2={0.1: 0.751, 0.2: 0.827, 0.3: 0.867, 0.4: 0.886, 0.5: 0.885, 0.6: 0.879, 0.7: 0.859, 0.8: 0.816, 0.9: 0.723}
bestAccuracy_dict3={0.1: 0.799, 0.2: 0.851, 0.3: 0.874, 0.4: 0.887, 0.5: 0.898, 0.6: 0.89, 0.7: 0.868, 0.8: 0.83, 0.9: 0.747}

In [21]:
def plot_mle(dict1, dict2, dict3):
    # extract keys and values from the dictionaries and keep them into lists
    x = []
    y = []
    x1 = []
    y1 = []
    x2 = []
    y2 = []

    for data in dict1.keys():
        x.append(data)
    for data in dict1.values():
        y.append(data)

    for data in dict2.keys():
        x1.append(data)
    for data in dict2.values():
        y1.append(data)

    for data in dict3.keys():
        x2.append(data)
    for data in dict3.values():
        y2.append(data)
        
    # plot accuracy with respect to prior probability of class0 
    plt.figure(figsize=(10,8))
    line1 = plt.plot(x, y,'ko-',label='Case I')
    line2 = plt.plot(x1, y1,'ro-',label='Case II') 
    line3 = plt.plot(x2, y2,'mo-',label='Case III')
    plt.legend(loc='upper left')
    plt.xlabel('Prior probabilities for Class1')
    plt.ylabel('Accuracy')
    plt.title('Performance Analysis for Different Prior Probability Distributions')
    
# uncomment this line for output
# plot_mle (bestAccuracy_dict1,bestAccuracy_dict2,bestAccuracy_dict3)

In [ ]:
### Parameter Estimation using Bayesian Learning for Whole Sample
def bayesian_learning_all(trainFile):
    # read the trainset file     
    fileData = open(trainFile, "r",encoding='utf-8-sig')
    lines = fileData.readlines()
    
    xsClass = []
    ysClass = []
    for line in lines:
        x = line.split()
        xs = float(x[0])
        ys = float(x[1])
        yc = int(x[2])
        xsClass.append(xs)
        ysClass.append(ys)
        
    matClass = np.zeros((len(xsClass), 2))
    for i in range(matClass.shape[0]):
        matClass[i][0] = xsClass[i]
    for i in range(matClass.shape[0]):
        matClass[i][1] = ysClass[i]

    # sample mean and covariance
    mean = np.mean(matClass,axis=0)
    meanClass = np.matrix(mean).T
    
    sigmaClass = np.zeros((len(xsClass), 2))
    cov = np.zeros((len(xsClass), 2))
    for i in range(matClass.shape[0]):
        cov[i][0] = matClass[i][0]-mean[0]
        cov[i][1] = matClass[i][1]-mean[1]
        transposecov = np.matrix(cov).T
        sigmaClass = transposecov*cov/matClass.shape[0]

    # guess values for mean and covariance
    meanGuess = np.matrix('0.01 0.44').T
    sigmaGuess = np.matrix('0.2 0.02;0.02 0.06')
    
    # find mean vector 
    n = len(matClass)
    bayesMeanPart1 = sigmaGuess*np.linalg.inv(sigmaGuess+(1/n)*sigmaClass)*meanClass
    bayesMeanPart2 = (1/n)*sigmaClass*np.linalg.inv(sigmaGuess+(1/n)*sigmaClass)*meanGuess
    bayesMean = bayesMeanPart1+bayesMeanPart2

    # find covariance
    bayesVar = sigmaGuess*np.linalg.inv(sigmaGuess+(1/n)*sigmaClass)*(1/n)*sigmaClass
    
    return bayesMean, bayesVar

# uncomment for output
# bayesian_learning_all ("SYNTH.TR")

In [ ]:
def bayesian_learning(trainFile):
    # read the trainset file     
    fileData = open(trainFile, "r",encoding='utf-8-sig')
    lines = fileData.readlines()
    
    # create two matrices for two different classes    
    xsClass1 = []
    ysClass1 = []
    xsClass2 = []
    ysClass2 = []

    for line in lines:
        x = line.split()
        xs = float(x[0])
        ys = float(x[1])
        yc = int(x[2])
        if yc == 0:
            xsClass1.append(xs)
            ysClass1.append(ys)
        else:
            xsClass2.append(xs)
            ysClass2.append(ys)
     
    # for class0
    matClass1 = np.zeros((len(xsClass1), 2))
    for i in range(matClass1.shape[0]):
        matClass1[i][0] = xsClass1[i]
    for i in range(matClass1.shape[0]):
        matClass1[i][1] = ysClass1[i]
    
    # for class1
    matClass2 = np.zeros((len(xsClass2), 2))
    for i in range(matClass2.shape[0]):
        matClass2[i][0] = xsClass2[i]
    for i in range(matClass2.shape[0]):
        matClass2[i][1] = ysClass2[i]

    # sample mean for class0
    mean1 = np.mean(matClass1,axis=0)
    meanClass1 = np.matrix(mean1).T
    
    # sample mean for class1
    mean2 = np.mean(matClass2,axis=0)
    meanClass2 = np.matrix(mean2).T

    # sample covariance for class0
    sigmaClass1 = np.zeros((len(xsClass1), 2))
    cov1 = np.zeros((len(xsClass1), 2))
    for i in range(matClass1.shape[0]):
        cov1[i][0] = matClass1[i][0]-mean1[0]
        cov1[i][1] = matClass1[i][1]-mean1[1]
        transposecov1 = np.matrix(cov1).T
        sigmaClass1 = transposecov1*cov1/matClass1.shape[0]

    # sample covariance for class1
    sigmaClass2 = np.zeros((len(xsClass1), 2))
    cov2 = np.zeros((len(xsClass1), 2))
    for i in range(matClass2.shape[0]):
        cov2[i][0] = matClass2[i][0]-mean2[0]
        cov2[i][1] = matClass2[i][1]-mean2[1]
        transposecov2 = np.matrix(cov2).T
        sigmaClass2 = transposecov2*cov2/matClass2.shape[0]
        
    # guess values for mean and covariance
    meanGuess1 = np.matrix('0.01 0.44').T
    sigmaGuess1 = np.matrix('0.02 0.002;0.002 0.006')
    meanGuess2 = np.matrix('0.02 0.3').T
    sigmaGuess2 = np.matrix('0.02 0.01;0.01 0.05')
    
    n = len(matClass1)
    commonPart1 = sigmaGuess1+(1/n)*sigmaClass1
    commonPart2 = sigmaGuess2+(1/n)*sigmaClass2

    # find mean vector for class0
    bayesMeanPart1 = sigmaGuess1*np.linalg.inv(commonPart1)*meanClass1
    bayesMeanPart2 = (1/n)*sigmaClass1*np.linalg.inv(commonPart1)*meanGuess1
    bayesMean1 = bayesMeanPart1+bayesMeanPart2

    # find mean vector for class1
    bayesMeanPart1a = sigmaGuess2*np.linalg.inv(commonPart2)*meanClass2
    bayesMeanPart2a = (1/n)*sigmaClass2*np.linalg.inv(commonPart2)*meanGuess2
    bayesMean2 = bayesMeanPart1a+bayesMeanPart2a

    # find covariance for class 0
    bayesVar1 = sigmaGuess1*np.linalg.inv(commonPart1)*(1/n)*sigmaClass1
    
    # find covariance for class 1
    bayesVar2 = sigmaGuess2*np.linalg.inv(commonPart2)*(1/n)*sigmaClass2
    
    return bayesMean1, bayesMean2, bayesVar1, bayesVar2

# uncomment for output
# bayesian_learning ("SYNTH.TR")

In [ ]:
bayesMean1,bayesMean2,bayesVar1,bayesVar2  = bayesian_learning("SYNTH.TR")
# print (bayesian_learning("SYNTH.TR"))

# calculate for different probabilities and create dictionary with probability as keys and accuracy as values
def mpp(testFile, probClass1,mean1,mean2,sigma1,sigma2,sigmaSqr):
    probClass2 = 1-probClass1
    
    # read the test data file
    fileData1 = open(testFile, "r",encoding='utf-8-sig')
    liness = fileData1.readlines()
    
    # create a matrix from test data for calculation
    testSet = np.zeros((len(liness),3))
    xst = []
    yst = []
    yct = []
    for line in liness:
        xt = line.split()
        xst.append(float(xt[0]))
        yst.append(float(xt[1]))
        yct.append(int(xt[2]))
    for i in range(len(liness)):
        testSet[i][0] = xst[i]
        testSet[i][1] = yst[i]
        testSet[i][2] = yct[i]
        
    # find covariance matrices for CASE I
    I = np.identity(2)
    sigma3 = sigmaSqr*I

    # calculate determinant and inverse of new covariance Matrix (case 1: covariance matrices are equal to (sigma^2)I)
    detClass1 = np.linalg.det(sigma3)
    detClass2 = detClass1
    invClass1 = np.linalg.inv(sigma3)
    invClass2 = invClass1

    # test the three cases of discriminant analysis for best accuracy
    LAMBDA11 = 0
    LAMBDA12 = 1
    LAMBDA21 = 1
    LAMBDA22 = 0

    testCase = 0
    while testCase < 3:
        bestLambda1 = 1.0
        bestLambda2 = 1.0
        bestAccuracy = 0

        lambdaClass1 = 1.0
        while lambdaClass1 > 0:
            bestLambda1 = 1.0
            bestLambda2 = 1.0
            bestAccuracy = 0

            lambdaClass2 = 1.0
            while lambdaClass2 > 0:
                predictedMatrix = np.zeros((len(testSet), 2))
                correctGuesses = 0
                line1 = np.zeros((1, 2))
                for i in range(len(testSet)):
                    line1[:,0] = testSet[i][0]
                    line1[:,1] = testSet[i][1]
                    line2 = line1.T
                    mahalanobis = line2 - mean1
                    mahalanobis1 = mahalanobis.T
                    mahalanobis2 = mahalanobis1 * invClass1
                    mahalanobis3 = mahalanobis2 * (line2 - mean1)
                    varMahalanobis = float(-0.5 * mahalanobis3[0][0])
                    probIsClass1 = float((1.0 / math.sqrt(2 * math.pi * detClass1)) * math.exp(varMahalanobis) * (probClass1))

                    mahalanobiss = (line2 - mean2)
                    mahalanobiss1 = mahalanobiss.T
                    mahalanobiss2 = mahalanobiss1 * invClass2
                    mahalanobiss3 = mahalanobiss2 * (line2 - mean2)
                    varMahalanobiss = float(-0.5 * mahalanobiss3[0][0])

                    probIsClass2 = float((1.0 / math.sqrt(2 * math.pi * detClass2)) * math.exp(varMahalanobiss) * (probClass2))

                    aux = float(probIsClass1)
                    probIsClass1 = lambdaClass1 * probIsClass1 + (1 - lambdaClass1) * probIsClass2
                    probIsClass2 = lambdaClass2 * probIsClass2 + (1 - lambdaClass2) * aux

                    predictedClass = 0
                    if probIsClass2 > probIsClass1:
                        predictedClass = 1

                    if predictedClass == testSet[i][2]: 
                        correctGuesses+=1

                    error = float(min(probIsClass2, probIsClass1))

                    predictedMatrix[i][1] = error
                    predictedMatrix[i][0] = predictedClass

                acc = float(correctGuesses) / float(len(testSet))
                if acc > bestAccuracy:
                    bestAccuracy = acc
                    bestLambda1 = lambdaClass1
                    bestLambda2 = lambdaClass2
                lambdaClass2 -= 0.05
            lambdaClass1 -= 0.05

        if testCase == 0:
            print ("Discriminant Function Case 1 (features independents): accuracy = ",bestAccuracy,
                   "with probability",probClass1,"and",probClass2, "Lambda1,1=",bestLambda1,";Lambda1,2=",
                   1 - bestLambda1,";Lambda2,1=",bestLambda2,";Lambda2,2=",(1 - bestLambda2))

            # calculate determinant and inverse of new covariance matrices (case 2: covariance matrices is equal)
            detClass1 = np.linalg.det(sigma1)
            detClass2 = detClass1
            invClass1 = np.linalg.inv(sigma1)
            invClass2 = invClass1

        elif testCase == 1:
            print ("Discriminant Function Case 2 (covariance matrix equal for all features): accuracy = ",
                   bestAccuracy,"with probability",probClass1,"and",probClass2," Lambda1,1=",bestLambda1,
                   ";Lambda1,1=",bestLambda1,";Lambda1,2=",1-bestLambda1,";Lambda2,1=",bestLambda2,";Lambda2,2=",
                   (1 - bestLambda2))

            # calculate determinant and inverse of new covariance matrices (case 3: covariance matrices are different)
            detClass1 = np.linalg.det(sigma1)
            invClass1 = np.linalg.inv(sigma1)
            invClass2 = np.linalg.inv(sigma2)
            detClass2 = np.linalg.det(sigma2)

        elif testCase == 2:
            print ("Discriminant Function Case 3 (arbitrary): accuracy = ", bestAccuracy,
                   "with probability",probClass1,"and",probClass2," Lambda1,1=",bestLambda1 , ";Lambda1,2=" 
                   , (1 - bestLambda1),";Lambda2,1=" , bestLambda2 ,";Lambda2,2=", (1 - bestLambda2))

        testCase+=1

# uncomment for output
# mpp("SYNTH.TE", 0.5,bayesMean1,bayesMean2,bayesVar1,bayesVar2,bayesVar2[0,0])

In [ ]:
# create dictionary for plotting
Bayesian_dict = {'Case I':0.723 , 'Case II': 0.888,'Case III':0.89 }
MLE_dict = {'Case I': 0.713,'Case II':0.885,'Case III':0.89}

In [ ]:
def plot_bayes_mle(dict1, dict2):
    # extract keys and values from the dictionaries and keep them into lists
    x = []
    y = []
    x1 = []
    y1 = []

    for data in dict1.keys():
        x.append(data)
    for data in dict1.values():
        y.append(data)

    for data in dict2.keys():
        x1.append(data)
    for data in dict2.values():
        y1.append(data)
        
    # plot accuracy with respect to 3 cases 
    plt.clf()
    fig, axs = plt.subplots(1,1,figsize=(8,5))
    opacity = .7
    w = .3
    
    a1 = [x for x, _ in enumerate(x)]
    b1 = [x + w for x in a1]

    axs.bar(a1, y, color='red', width=w, edgecolor='white', label='Bayesian Learnning')
    axs.bar(b1, y1, color='yellowgreen', width=w, edgecolor='white', label='MLE')

    plt.xticks([x + w for x in range(len(x))], x)
    plt.legend(loc='upper left')
    plt.xlabel('Three Cases of Discriminant Analysis')
    plt.ylabel('Accuracy')
    plt.title('Performance Analysis of Bayesian Learning & MLE with Prior Probability 0.5 Each')
    plt.tight_layout()
    plt.show()

# uncomment for output
# plot_bayes_mle (Bayesian_dict,MLE_dict)

In [ ]:
def bimodal(testFile, probClass1, a1):
    fileData1 = open(testFile, "r",encoding='utf-8-sig')
    liness = fileData1.readlines()
    
    # create a matrix from test data for calculation
    testSet = np.zeros((len(liness),3))
    xst = []
    yst = []
    yct = []
    for line in liness:
        xt = line.split()
        xst.append(float(xt[0]))
        yst.append(float(xt[1]))
        yct.append(int(xt[2]))
    for i in range(len(liness)):
        testSet[i][0] = xst[i]
        testSet[i][1] = yst[i]
        testSet[i][2] = yct[i]
    
    mean1 = np.matrix('-.75 .2')
    mean2 = np.matrix('-.23 .79')

    sigmaClass1 = np.matrix('0.25 0; 0 0.3')
    sigmaClass2 = np.matrix('.07 .021 ; .021 0.047')


    meanex = np.matrix('0.3 0.3')                #extra mean for class 0 
    covex = np.matrix('.1 0 ; 0 0.01')    #extra variance for class 0

    meanex1 = np.matrix('0.55 0.69')               #extra mean for class 1
    covex1 = np.matrix('.049 .055; .055 .089')   #extra variance for class 1

    I = np.identity(2)
    sigma3 = sigmaClass1[0,0]*I
    
    # calculate determinant and inverse of new covariance Matrix (case 1: covariance matrices are equal to (sigma^2)I)
    detClass1 = np.linalg.det(sigma3)
    detClass2 = detClass1
    invClass1 = np.linalg.inv(sigma3)
    invClass2 = invClass1

    # test the three cases of discriminant analysis for best accuracy
    LAMBDA11 = 0
    LAMBDA12 = 1
    LAMBDA21 = 1
    LAMBDA22 = 0

    d = 2
    a2 = 1-a1
    probClass2 = 1-probClass1


    detClass1 = np.linalg.det(sigma3)
    detClass2 = detClass1
    detcovex = detClass1
    detcovex1 = detClass1

    invClass1 = np.linalg.inv(sigma3)
    invClass2 = invClass1
    invcovex = invClass1
    invcovex1 = invClass1


    testCase = 0
    while testCase < 3:
        bestLambda1 = 1.0
        bestLambda2 = 1.0
        bestAccuracy = 0

        lambdaClass1 = 1.0
        while lambdaClass1 > 0:
            bestLambda1 = 1.0
            bestLambda2 = 1.0
            bestAccuracy = 0

            lambdaClass2 = 1.0
            while lambdaClass2 > 0:
                predictedMatrix = np.zeros((len(testSet), 2))
                correctGuesses = 0
                line1 = np.zeros((1, 2))
                for i in range(len(testSet)):
                    line1[:,0] = testSet[i][0]
                    line1[:,1] = testSet[i][1]
                    line2 = line1.T
                    mahalanobis = line2 - (mean1).T
                    mahalanobis1 = mahalanobis.T
                    mahalanobis2 = mahalanobis1 * invClass1
                    mahalanobis3 = mahalanobis2 * (line2 - (mean1).T)
                    varMahalanobis = float(-0.5 * mahalanobis3[0][0])

                    mahalanobisex = line2 - (meanex).T
                    mahalanobis1ex = mahalanobisex.T
                    mahalanobis2ex = mahalanobis1ex * invcovex
                    mahalanobis3ex = mahalanobis2ex * (line2 - (meanex).T)
                    varMahalanobisx = float(-0.5 * mahalanobis3ex[0][0])

                    # bimodal pdf
                    probIsClass1 = float(a1*(1.0 / (((2 * math.pi)**(d/2) )* math.sqrt(detClass1))) * math.exp(varMahalanobis) * (probClass1) +
                                        a2*(1.0 / (((2 * math.pi)**(d/2)) * math.sqrt(detcovex))) * math.exp(varMahalanobisx) * (probClass1))


                    mahalanobiss = line2 - (mean2).T
                    mahalanobiss1 = mahalanobiss.T
                    mahalanobiss2 = mahalanobiss1 * invClass2
                    mahalanobiss3 = mahalanobiss2 * (line2 - (mean2).T)
                    varMahalanobiss = float(-0.5 * mahalanobiss3[0][0])

                    mahalanobisex1 = line2 - (meanex1).T
                    mahalanobis1ex1 = mahalanobisex1.T
                    mahalanobis2ex1 = mahalanobis1ex1 * invcovex1
                    mahalanobis3ex1 = mahalanobis2ex1 * (line2 - (meanex1).T)
                    varMahalanobisx1 = float(-0.5 * mahalanobis3ex1[0][0])

                    probIsClass2 = float(a1*(1.0 / (((2 * math.pi)**(d/2)) * math.sqrt(detClass2))) * math.exp(varMahalanobiss) * (probClass2) +
                                        a2*(1.0 / (((2 * math.pi)**(d/2)) * math.sqrt(detcovex1))) * math.exp(varMahalanobisx1) * (probClass2))

                    aux = float(probIsClass1)
                    probIsClass1 = lambdaClass1 * probIsClass1 + (1 - lambdaClass1) * probIsClass2
                    probIsClass2 = lambdaClass2 * probIsClass2 + (1 - lambdaClass2) * aux

                    predictedClass = 0
                    if probIsClass2 > probIsClass1:
                        predictedClass = 1

                    if predictedClass == testSet[i][2]: 
                        correctGuesses+=1

                    error = float(min(probIsClass2, probIsClass1))

                    predictedMatrix[i][1] = error
                    predictedMatrix[i][0] = predictedClass

                acc = float(correctGuesses) / float(len(testSet))
                if acc > bestAccuracy:
                    bestAccuracy = acc
                    bestLambda1 = lambdaClass1
                    bestLambda2 = lambdaClass2
                lambdaClass2 -= 0.05
            lambdaClass1 -= 0.05

        if testCase == 0:
            print ("Discriminant Function Case 1 (features independents): accuracy = ",bestAccuracy,
                   "with probability",probClass1,"and",probClass2, "Lambda1,1=",
                   bestLambda1,";Lambda1,2=",1 - bestLambda1,";Lambda2,1=",bestLambda2,";Lambda2,2=",
                   (1 - bestLambda2))
            acc1 = bestAccuracy

            detClass1 = np.linalg.det(covex)
            detClass2 = detClass1
            detcovex = detClass1
            detcovex1 = detClass1
            invClass1 = np.linalg.inv(covex)
            invClass2 = invClass1
            invcovex = invClass1
            invcovex1 = invClass1

        elif testCase == 1:
            print ("Discriminant Function Case 2 (covariance matrix equal for all features): accuracy = ",
                   bestAccuracy,"with probability",probClass1,"and",probClass2,
                   " Lambda1,1=",bestLambda1,";Lambda1,2=",
                   1-bestLambda1,";Lambda2,1=",bestLambda2,";Lambda2,2=",(1 - bestLambda2))
            acc2 = bestAccuracy

            detClass1 = np.linalg.det(sigmaClass1)
            invClass1 = np.linalg.inv(sigmaClass1)
            invClass2 = np.linalg.inv(sigmaClass2)
            detClass2 = np.linalg.det(sigmaClass2)
            detcovex = np.linalg.det(covex)
            invcovex = np.linalg.inv(covex)
            detcovex1 = np.linalg.det(covex1)
            invcovex1 = np.linalg.inv(covex1)


        elif testCase == 2:
            print ("Discriminant Function Case 3 (arbitrary): accuracy = ", bestAccuracy,
                   "with probability",probClass1,"and",probClass2," Lambda1,1=",
                   bestLambda1 , ";Lambda1,2=" , (1 - bestLambda1),";Lambda2,1=" , bestLambda2 , 
                   ";Lambda2,2=", (1 - bestLambda2))
            acc3 = bestAccuracy

        testCase+=1

# uncomment this line for output
# bimodal('SYNTH.TE',0.5, 0.8)

In [ ]:
# create dictionary for plotting
bimodal_dict = {'Case I': 0.777, 'Case II': 0.886,'Case III': 0.9}
unimodal_dict = {'Case I': 0.713,'Case II':0.885,'Case III':0.898}

In [ ]:
def plot_modal(dict1, dict2):
    # extract keys and values from the dictionaries and keep them into lists
    x = []
    y = []
    x1 = []
    y1 = []

    for data in dict1.keys():
        x.append(data)
    for data in dict1.values():
        y.append(data)

    for data in dict2.keys():
        x1.append(data)
    for data in dict2.values():
        y1.append(data)
        
    # plot accuracy with respect to 3 cases 
    plt.clf()
    fig, axs = plt.subplots(1,1,figsize=(8,5))
    opacity = .7
    w = .3
    
    a1 = [x for x, _ in enumerate(x)]
    b1 = [x + w for x in a1]

    axs.bar(a1, y, color='red', width=w, edgecolor='white', label='Bimodal')
    axs.bar(b1, y1, color='yellowgreen', width=w, edgecolor='white', label='One modal')

    plt.xticks([x + w for x in range(len(x))], x)
    plt.legend(loc='upper left')
    plt.xlabel('Three Cases of Discriminant Analysis')
    plt.ylabel('Accuracy')
    plt.title('Performance Analysis of Bimodal & One modal Gaussian Distributions \n with Prior Probability 0.5 Each')
    plt.show()

# uncomment this line for output
# plot_modal (bimodal_dict,unimodal_dict)